In [1]:
import pandas as pd
import os
import openai
from IPython.display import display, Markdown

In [ ]:
# openai.api_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

In [ ]:
# Set openai.api_key to the OPENAI environment variable
openai.api_key = os.environ["OPENAI"]

In [6]:
df = pd.read_json(path_or_buf="nope_data/nli_corpus.main.jsonl", lines=True)
df.to_csv("nope_data/data.csv", index=False)

In [ ]:
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo", 
  messages = [{"role": "system", "content" : "You are ChatGPT, a large language model trained by OpenAI. Answer as concisely as possible.\nKnowledge cutoff: 2021-09-01\nCurrent date: 2023-03-02"},
{"role": "user", "content" : "How are you?"},
{"role": "assistant", "content" : "I am doing well"},
{"role": "user", "content" : "What is the mission of the company OpenAI?"}]
)
#print(completion)
print(completion)